# Occupancy Distribution Through Time.

Let's gain some insights about the distribution of occupancy count across time and locations.

## Script setup

In [8]:
# The Python interpreter requires this workaround to
# import modules outside of this notebook's directory.
import sys
sys.path.append("..")

In [9]:
# Third-party and standard modules
from typing import Optional
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import pytz
from itertools import repeat
from importlib import reload

# User-defined modules.
# The reload() function is needed to update modules after
# changes are made to their files.
import dataframe_manip as dfm
dfm = reload(dfm)

ImportError: attempted relative import with no known parent package

In [ ]:
def dataframe_info(d: pd.DataFrame) -> str:
    """A basic function for getting dataframe info."""
    return """
    shape: {}
    index datatype: {}
    na value count: {}
    """.format(
        str(d.shape),
        d.index.dtype,
        d.isna().sum().sum()
    )

## Script parameters

The following values are this script's parameters.

In [ ]:
filepath = '../wifi_data_until_20190204.csv'
timezone = pytz.timezone('US/Pacific')

## The main dataframe: `occupancy`

We'll be mainly working with a single dataframe named `occupancy` created from the csv provided.

`occupancy` is a time-series dataframe. Each of its rows represents a point in time. Each of its columns represents a WiFi access point.

In [ ]:
occupancy: pd.DataFrame = dfm.csv_to_timeseries_df(
    filepath=filepath, timezone=timezone
)
print(dataframe_info(occupancy))

These data were acquired through the SNMP protocol, which doesn't record data for moments of 0 connected devices. This means that NA values in our dataframe could represent either 0's or truly missing data.

However, the missing data is mostly (if not all) due to the fact that we started and stopped collecting data from different access points at different times. We rarely (if ever) returned to an access point after stopping.

This allows us to assume that the first and last uninterrupted streams of NA values are missing data while **intervening NA values** are actually 0's. We will fill these **intervening NA values** with 0.

In [ ]:
dfm.fill_intervening_nas(
    occupancy, inplace=True, fill_val = 0
)
print(dataframe_info(occupancy))

## Occupancy by the hour

Let's start by finding out how total connectivity (across all access points) varies by the hour.

We'll reduce `data` to the `total_occupancy_vs_time` dataframe. Like `data`, each of its rows represents a point in time. However, it only has one column representing all access points.

In [ ]:
total_occupancy_vs_time = dfm.row_totals(occupancy)

print(dataframe_info(
    total_occupancy_vs_time
))

Let's create create 24 boxplots --each representing 1 hour-- which show how the distribution of **occupancy** (as measured by the number of devices connected to access points) *throughout all buildings* varies hour-by-hour.

In [ ]:
fig, ax = plt.subplots(figsize=(24, 15))

seaborn.boxplot(
    # The hours (a number [0,23]) for each row.
    x=total_occupancy_vs_time.index.hour,
    # The total occupancy at that hour.
    y=total_occupancy_vs_time,
    ax=ax
)

It's immediately clear that measures of occupancy total have strong relationships with the hour of the day. The transitions between hours are smooth (i.e. continuous) and aggressive.

The total **peaks around 14:30 (2:30 pm)** and is **lowest around 6:30**.

Interestingly, the **interquartile range** (i.e. rectangle length) in occupancy also **peaks around 2:30** and is **lowest around 6:30**. The upper quartile (75th percentile) moves far more aggressively than the lower quartile or median. This may be due to a fixed/stable number of devices which don't vary much throughout the day. These could be public devices (e.g. library computers) or staff devices (e.g. staff phones). It's worth noting that the minimum doesn't change much throughout the day.

In [ ]:
# acpt is short for 'access point'
stats_per_acpt: pd.DataFrame = pd.DataFrame.from_dict({
    'total':
    dfm.column_totals(occupancy),
    'mean':
    dfm.column_means(occupancy, skipna=True),
    'mean, skipna=False':
    dfm.column_means(occupancy, skipna=False),
    'median':
    dfm.column_medians(occupancy, skipna=True),
    'median, skipna=False':
    dfm.column_medians(occupancy, skipna=False)
})

print(dataframe_info(
    stats_per_acpt
))

In [ ]:
plt.scatter(
    repeat(
        range(0, stats_per_acpt.shape[1]), 2
    ),
    pd.concat(
        [stats_per_acpt['mean'], stats_per_acpt['mean, skipna=False']],
        ignore_index=True
    ),
    np.concatenate(
        (np.tile([255, 0, 0], dtype=np.ubyte), np.tile([0, 0, 255], dtype=np.ubyte)),
        axis=0
    )
)
plt.show()